In [1]:
""" This is a separated jupyter notebook file specific for the analysis of Libraries entites in 'luoghi della cultura dataset'."""

from rdflib import Graph
import csv
import pandas as pd
from IPython.display import display

# Load the  datastet luoghi della cultura RDF file
rdf_file =  r"C:\Users\HWRUser\Desktop\DHDK Course\Information Visualization\Data_Italian_publishers\dataset_luoghi_cultura.rdf"
g = Graph()
g.parse(rdf_file, format="xml") 


#Exploratory Query for identifying all the libraries IDs, Names, Cities, Regions, Latitute and Longitude

query_1 = """
 PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
 PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
 PREFIX cis: <http://dati.beniculturali.it/cis/>
 PREFIX clvapit: <https://w3id.org/italia/onto/CLV/>
 PREFIX dc: <http://purl.org/dc/elements/1.1/>
 PREFIX geo1: <http://www.w3.org/2003/01/geo/wgs84_pos#>

 SELECT ?library ?librarylabel ?libraryCity ?libraryRegion ?librarylatitude ?librarylongitude 
 WHERE {

   {
    ?library a cis:Library .
  }
  UNION
  {
    ?library dc:type "Biblioteca Statale" .
  }

   ?library a cis:CulturalInstituteOrSite.
   ?library rdfs:label ?librarylabel.
   ?library cis:hasSite ?librarysite.
   ?librarysite cis:siteAddress ?libraryAddress.
   ?libraryAddress clvapit:hasRegion ?libraryRegion.
   ?libraryAddress clvapit:hasCity ?libraryCity.
   ?library geo1:lat ?librarylatitude.
   ?library geo1:long ?librarylongitude.
 } 
 
"""


# Perform the query
results = g.query(query_1)

#Libraries Dataframe
data_Libraries = []
for row in results:
    data_Libraries.append({'Library_ID': str(row['library']), 'Library_Name': str(row['librarylabel']),'Library_City': str(row['libraryCity']), 'Library_Region': str(row['libraryRegion']),'Library_Latitude': str(row['librarylatitude']), 'Library_Longitude': str(row['librarylongitude'])})
df_Libraries = pd.DataFrame(data_Libraries)

# List of IDs to remove that are not libraries: some of the entity libraries were classified in the dataset as libraries but after an in-depth search it was find out that they were not exactly libraries. For deciding if it was the case these were compared with the same entity identified in WikiData.
IDs_to_remove=["http://dati.beniculturali.it/mibact/luoghi/resource/CulturalInstituteOrSite/102857",
               "http://dati.beniculturali.it/mibact/luoghi/resource/CulturalInstituteOrSite/102889",
               "http://dati.beniculturali.it/mibact/luoghi/resource/CulturalInstituteOrSite/102891",
               "http://dati.beniculturali.it/mibact/luoghi/resource/CulturalInstituteOrSite/101975",
               "http://dati.beniculturali.it/mibact/luoghi/resource/CulturalInstituteOrSite/102879",
               "http://dati.beniculturali.it/mibact/luoghi/resource/CulturalInstituteOrSite/102917",
               "http://dati.beniculturali.it/mibact/luoghi/resource/CulturalInstituteOrSite/102858",
               "http://dati.beniculturali.it/mibact/luoghi/resource/CulturalInstituteOrSite/102876",
               "http://dati.beniculturali.it/mibact/luoghi/resource/CulturalInstituteOrSite/102853",
               "http://dati.beniculturali.it/mibact/luoghi/resource/CulturalInstituteOrSite/110257",
               "http://dati.beniculturali.it/mibact/luoghi/resource/CulturalInstituteOrSite/102864",
               "http://dati.beniculturali.it/mibact/luoghi/resource/CulturalInstituteOrSite/102855"]

# Find rows to remove in Library_ID
rows_to_remove = df_Libraries[df_Libraries['Library_ID'].isin(IDs_to_remove)].index
library_df = df_Libraries.drop(rows_to_remove)
library_df.reset_index(drop=True, inplace=True)


#Dataframe for counting number of libraries for region and the total number
library_region_counts = library_df.groupby('Library_Region').size().reset_index(name='Count')
# Calculate the total count
total_library_region_count = library_region_counts['Count'].sum()
# Create a DataFrame for the total row
total_row = pd.DataFrame({'Library_Region': ['TOTAL'], 'Count': [total_library_region_count]})
# Concat the total row to the library region count dataframe
library_region_counts= pd.concat([library_region_counts, total_row], ignore_index=True)


# #Dataframes for counting number of libraries for city and the total number
library_city_counts = library_df.groupby('Library_City').size().reset_index(name='Count')
total_library_city_count = library_city_counts['Count'].sum()
# Create a DataFrame for the total row
total_row = pd.DataFrame({'Library_City': ['TOTAL'], 'Count': [total_library_city_count]})
# Concat the total row dataframe to the library city count dataframe
library_city_counts= pd.concat([library_city_counts, total_row], ignore_index=True)

#Displaying library DataFrame, library region and library city count dataframes with total count
display(library_df)
display(library_region_counts)
display(library_city_counts)

# CSV files from all Dataframes for libraries
csv_file_libraries = "Libraries_Luoghi_Cultura.csv"
library_df.to_csv(csv_file_libraries, index=False, encoding='utf-8')
print(f"Results have been written to {csv_file_libraries}")

csv_file_library_region_count = "Libraries_Region_Count_Luoghi_Cultura.csv"
library_region_counts.to_csv(csv_file_library_region_count, index=False, encoding='utf-8')
print(f"Results have been written to {csv_file_library_region_count}")

csv_file_library_city_count = "Libraries_City_Count_Luoghi_Cultura.csv"
library_city_counts.to_csv(csv_file_library_city_count, index=False, encoding='utf-8')
print(f"Results have been written to {csv_file_library_city_count}")

,Library_ID,Library_Name,Library_City,Library_Region,Library_Latitude,Library_Longitude
0,http://dati.beniculturali.it/mibact/luoghi/res...,Centro dantesco dei Frati Minori Conventuali,http://dati.beniculturali.it/mibact/luoghi/res...,http://dati.beniculturali.it/mibact/luoghi/res...,44.41629,12.200871
1,http://dati.beniculturali.it/mibact/luoghi/res...,Biblioteca 'Mario Gromo',http://dati.beniculturali.it/mibact/luoghi/res...,http://dati.beniculturali.it/mibact/luoghi/res...,45.081963,7.681357
2,http://dati.beniculturali.it/mibact/luoghi/res...,Biblioteca comunale,http://dati.beniculturali.it/mibact/luoghi/res...,http://dati.beniculturali.it/mibact/luoghi/res...,41.63192,13.71498
3,http://dati.beniculturali.it/mibact/luoghi/res...,Biblioteca comunale - Museo,http://dati.beniculturali.it/mibact/luoghi/res...,http://dati.beniculturali.it/mibact/luoghi/res...,40.184994,18.35104
4,http://dati.beniculturali.it/mibact/luoghi/res...,Biblioteca Classense,http://dati.beniculturali.it/mibact/luoghi/res...,http://dati.beniculturali.it/mibact/luoghi/res...,44.414177,12.200108
...,...,...,...,...,...,...
56,http://dati.beniculturali.it/mibact/luoghi/res...,Biblioteca Universitaria di Sassari,http://dati.beniculturali.it/mibact/luoghi/res...,http://dati.beniculturali.it/mibact/luoghi/res...,40.722702,8.563728
57,http://dati.beniculturali.it/mibact/luoghi/res...,Musei Reali - Biblioteca Reale,http://dati.beniculturali.it/mibact/luoghi/res...,http://dati.beniculturali.it/mibact/luoghi/res...,45.07285,7.686345
58,http://dati.beniculturali.it/mibact/luoghi/res...,Biblioteca Statale Stelio Crise di Trieste,http://dati.beniculturali.it/mibact/luoghi/res...,http://dati.beniculturali.it/mibact/luoghi/res...,45.64559,13.763095
59,http://dati.beniculturali.it/mibact/luoghi/res...,San Matteo - Sede distaccata Biblioteca Univer...,http://dati.beniculturali.it/mibact/luoghi/res...,http://dati.beniculturali.it/mibact/luoghi/res...,43.7152,10.407801


,Library_Region,Count
0,http://dati.beniculturali.it/mibact/luoghi/res...,1
1,http://dati.beniculturali.it/mibact/luoghi/res...,1
2,http://dati.beniculturali.it/mibact/luoghi/res...,4
3,http://dati.beniculturali.it/mibact/luoghi/res...,5
4,http://dati.beniculturali.it/mibact/luoghi/res...,2
5,http://dati.beniculturali.it/mibact/luoghi/res...,19
6,http://dati.beniculturali.it/mibact/luoghi/res...,2
7,http://dati.beniculturali.it/mibact/luoghi/res...,4
8,http://dati.beniculturali.it/mibact/luoghi/res...,1
9,http://dati.beniculturali.it/mibact/luoghi/res...,4


,Library_City,Count
0,http://dati.beniculturali.it/mibact/luoghi/res...,1
1,http://dati.beniculturali.it/mibact/luoghi/res...,1
2,http://dati.beniculturali.it/mibact/luoghi/res...,1
3,http://dati.beniculturali.it/mibact/luoghi/res...,1
4,http://dati.beniculturali.it/mibact/luoghi/res...,1
5,http://dati.beniculturali.it/mibact/luoghi/res...,1
6,http://dati.beniculturali.it/mibact/luoghi/res...,1
7,http://dati.beniculturali.it/mibact/luoghi/res...,1
8,http://dati.beniculturali.it/mibact/luoghi/res...,1
9,http://dati.beniculturali.it/mibact/luoghi/res...,1


Results have been written to Libraries_Luoghi_Cultura.csv
Results have been written to Libraries_Region_Count_Luoghi_Cultura.csv
Results have been written to Libraries_City_Count_Luoghi_Cultura.csv


In [5]:
""" This part is dedicated to an in-depth analysing for identifying a specific type of libraries ("Biblioteca Statale")  whithin the 'luoghi della cultura' dataset. """

#Explorative query for identifying the Mibact libraries classified as dctype:"Biblioteca Statale"
query_1bis = """
 PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
 PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
 PREFIX cis: <http://dati.beniculturali.it/cis/>
 PREFIX clvapit: <https://w3id.org/italia/onto/CLV/>
 PREFIX dc: <http://purl.org/dc/elements/1.1/>
 PREFIX geo1: <http://www.w3.org/2003/01/geo/wgs84_pos#>

 SELECT ?library ?librarylabel ?libraryCity ?libraryRegion ?librarylatitude ?librarylongitude 
 WHERE {

   ?library dc:type "Biblioteca Statale" .
   ?library a cis:CulturalInstituteOrSite.
   ?library rdfs:label ?librarylabel.
   ?library cis:hasSite ?librarysite.
   ?librarysite cis:siteAddress ?libraryAddress.
   ?libraryAddress clvapit:hasRegion ?libraryRegion.
   ?libraryAddress clvapit:hasCity ?libraryCity.
   ?library geo1:lat ?librarylatitude.
   ?library geo1:long ?librarylongitude.
 } 
  

"""

# Perform the query
results = g.query(query_1bis)

#Libraries Dataframe
data_Libraries = []
for row in results:
    data_Libraries.append({'Library_ID': str(row['library']), 'Library_Name': str(row['librarylabel']),'Library_City': str(row['libraryCity']), 'Library_Region': str(row['libraryRegion']),'Library_Latitude': str(row['librarylatitude']), 'Library_Longitude': str(row['librarylongitude'])})
df_Libraries = pd.DataFrame(data_Libraries)

#List of IDs to remove that are not libraries: some of the entity libraries were classified in the dataset as libraries but after an in-depth search it was find out that they were not exactly libraries. For deciding if it was the case these were compared with the same entity identified in WikiData.
IDs_to_remove=["http://dati.beniculturali.it/mibact/luoghi/resource/CulturalInstituteOrSite/102857",
               "http://dati.beniculturali.it/mibact/luoghi/resource/CulturalInstituteOrSite/102889",
               "http://dati.beniculturali.it/mibact/luoghi/resource/CulturalInstituteOrSite/102891",
               "http://dati.beniculturali.it/mibact/luoghi/resource/CulturalInstituteOrSite/101975",
               "http://dati.beniculturali.it/mibact/luoghi/resource/CulturalInstituteOrSite/102879",
               "http://dati.beniculturali.it/mibact/luoghi/resource/CulturalInstituteOrSite/102917",
               "http://dati.beniculturali.it/mibact/luoghi/resource/CulturalInstituteOrSite/102858",
               "http://dati.beniculturali.it/mibact/luoghi/resource/CulturalInstituteOrSite/102876",
               "http://dati.beniculturali.it/mibact/luoghi/resource/CulturalInstituteOrSite/102853",
               "http://dati.beniculturali.it/mibact/luoghi/resource/CulturalInstituteOrSite/110257",
               "http://dati.beniculturali.it/mibact/luoghi/resource/CulturalInstituteOrSite/102864",
               "http://dati.beniculturali.it/mibact/luoghi/resource/CulturalInstituteOrSite/102855"]

# Find rows to remove in Library_ID
rows_to_remove = df_Libraries[df_Libraries['Library_ID'].isin(IDs_to_remove)].index
Mibact_libraries_df = df_Libraries.drop(rows_to_remove)
Mibact_libraries_df.reset_index(drop=True, inplace=True)

#Displaying Mibact Libraries Dataframe
display(Mibact_libraries_df)


#CSV files from Mibact Libraries Dataframe
csv_file_Mibact_libraries = "Mibact_Libraries_Luoghi_Cultura.csv"
Mibact_libraries_df.to_csv(csv_file_Mibact_libraries, index=False, encoding='utf-8')
print(f"Results have been written to {csv_file_Mibact_libraries}")

,Library_ID,Library_Name,Library_City,Library_Region,Library_Latitude,Library_Longitude
0,http://dati.beniculturali.it/mibact/luoghi/res...,Biblioteca nazionale Marciana - Sale monumenta...,http://dati.beniculturali.it/mibact/luoghi/res...,http://dati.beniculturali.it/mibact/luoghi/res...,45.434067,12.334472
1,http://dati.beniculturali.it/mibact/luoghi/res...,Biblioteca Nazionale di Cosenza,http://dati.beniculturali.it/mibact/luoghi/res...,http://dati.beniculturali.it/mibact/luoghi/res...,39.288624,16.26098
2,http://dati.beniculturali.it/mibact/luoghi/res...,Biblioteca Casanatense,http://dati.beniculturali.it/mibact/luoghi/res...,http://dati.beniculturali.it/mibact/luoghi/res...,41.898148,12.479639
3,http://dati.beniculturali.it/mibact/luoghi/res...,Biblioteca Statale di Lucca,http://dati.beniculturali.it/mibact/luoghi/res...,http://dati.beniculturali.it/mibact/luoghi/res...,43.8449,10.501189
4,http://dati.beniculturali.it/mibact/luoghi/res...,Biblioteca Riccardiana,http://dati.beniculturali.it/mibact/luoghi/res...,http://dati.beniculturali.it/mibact/luoghi/res...,43.77558,11.255349
5,http://dati.beniculturali.it/mibact/luoghi/res...,Biblioteca Medicea Laurenziana,http://dati.beniculturali.it/mibact/luoghi/res...,http://dati.beniculturali.it/mibact/luoghi/res...,43.774815,11.254537
6,http://dati.beniculturali.it/mibact/luoghi/res...,Biblioteca Statale del Monumento Nazionale di ...,http://dati.beniculturali.it/mibact/luoghi/res...,http://dati.beniculturali.it/mibact/luoghi/res...,41.76406,13.37067
7,http://dati.beniculturali.it/mibact/luoghi/res...,Biblioteca Medica Statale di Roma,http://dati.beniculturali.it/mibact/luoghi/res...,http://dati.beniculturali.it/mibact/luoghi/res...,0.040255,-0.010042
8,http://dati.beniculturali.it/mibact/luoghi/res...,Biblioteca Universitaria di Padova,http://dati.beniculturali.it/mibact/luoghi/res...,http://dati.beniculturali.it/mibact/luoghi/res...,45.40711,11.880526
9,http://dati.beniculturali.it/mibact/luoghi/res...,Biblioteca statale di Macerata,http://dati.beniculturali.it/mibact/luoghi/res...,http://dati.beniculturali.it/mibact/luoghi/res...,43.30105,13.448922


Results have been written to Mibact_Libraries_Luoghi_Cultura.csv
